In [ ]:
import os,sys
import mne
import utils  #my code
import json
import matplotlib.pyplot as plt
import numpy as np
import utils
%load_ext autoreload
%autoreload 2

if os.environ.get('DATA_DUSS') is not None:
    data_dir = os.path.expandvars('$DATA_DUSS')
else:
    data_dir = '/home/demitau/data'
    


#rawname_ = 'S01_off_hold'
#rawname_ = 'S01_on_hold'
#rawname_ = 'S01_off_move'
#rawname_ = 'S01_on_move'

rawname_ = 'S02_off_hold'
rawname_ = 'S02_on_hold'  

rawname_ = 'S03_off_move'  
rawname_ = 'S03_off_hold'  

rawname = rawname_ + '_resample_raw.fif'
fname_full = os.path.join(data_dir,rawname)
    

In [ ]:
# read file -- resampled to 256 Hz,  Electa MEG, EMG, LFP, EOG channels
raw = mne.io.read_raw_fif(fname_full, None)

In [ ]:
afterICA_name = rawname_ + '_resample_afterICA_raw.fif'
reconst_raw = mne.io.read_raw_fif(os.path.join(data_dir, afterICA_name), None)

In [ ]:
# Butterwordth 4th order -- for stopping power grid noise
#help(raw_sss.notch_filter)
raw.load_data()
import numpy as np
freqsToKill = np.arange(50, 128, 50) # harmonics of 50
raw.notch_filter(freqsToKill, picks=['meg','bio', 'emg'])

In [ ]:
# get info about bad MEG channels (from separate file)
with open('subj_info.json') as info_json:
        #raise TypeError

    #json.dumps({'value': numpy.int64(42)}, default=convert)
    gen_subj_info = json.load(info_json)
    
subj,medcond,task  = utils.getParamsFromRawname(rawname_)
badchlist = gen_subj_info[subj]['bad_channels'][medcond][task]
raw.info['bads'] = badchlist
print('bad channels are ',badchlist)

In [ ]:
raw.pick_types(meg=True, eog=True )
gen_subj_info[subj]['tremor_side']

In [ ]:
trem_times_fn = 'trem_times_tau.json'
with open(trem_times_fn ) as jf:
    trem_times_byhand = json.load(jf)   
trem_times_nms_fn = 'trem_times_tau_nms.json'
with open(trem_times_nms_fn ) as jf:
    trem_times_nms_byhand = json.load(jf)   

tremIntervalJan, artif         = utils.unpackTimeIntervals(trem_times_byhand, mainSide = True, 
                                                           gen_subj_info=gen_subj_info, skipNotLoadedRaws=0)
tremIntervalJan_nms, artif_nms = utils.unpackTimeIntervals(trem_times_nms_byhand, mainSide = False, 
                                                           gen_subj_info=gen_subj_info, skipNotLoadedRaws=0)
for rawn in [rawname_]:
    if rawn in artif_nms and rawn not in artif:
        artif[rawn] = artif_nms[rawn]
    else:
        if rawn in artif_nms:
            artif[rawn].update(artif_nms[rawn] )

In [ ]:
artif

In [ ]:
#Maxwell filter
do_Maxwell = 0
if do_Maxwell:
    fine_cal_file  = os.path.join(data_dir,  'sss_cal.dat')
    crosstalk_file = os.path.join(data_dir,  'ct_sparse.fif')
    raw_sss = mne.preprocessing.maxwell_filter(raw, cross_talk=crosstalk_file,
                                               calibration=fine_cal_file, coord_frame='meg')
else:
    raw_sss = raw.copy()

import gc; gc.collect()

In [ ]:
#high pass 1Hz, apparently it should help artifact removal
filt_raw = raw_sss.copy()
filt_raw = filt_raw.load_data().filter(l_freq=2., h_freq=None)  # advise: maybe 0.5

# get rid of bad channels
filt_raw.pick_types(meg=True, misc=False, bio=True, emg=True, eog=True, exclude='bads')

In [ ]:
subj,medcond,task  = utils.getParamsFromRawname(rawname_)
sind_str = subj

maintremside = gen_subj_info[subj]['tremor_side']
nonmaintremside = utils.getOppositeSideStr(maintremside)

In [ ]:
anns_fn = rawname_ + '_anns.txt'
anns_fn_full = os.path.join(data_dir, anns_fn)
anns = mne.read_annotations(anns_fn_full)
filt_raw.set_annotations(anns)
reconst_raw.set_annotations(anns)

ivalis = utils.ann2ivalDict(anns)

In [ ]:
sfreq = raw.info['sfreq']
#decim = 10
decim = 30
freq_min, freq_max = 3,90
freq_step = 1
freqs = np.arange(freq_min,freq_max,freq_step) #np.linspace(freq_min, freq_max, num=nfreqs)
endrec = raw.times[-1]

n_cycles = 7 + freqs  #/ 2.  # different number of cycle per frequency

In [ ]:
gc.collect()

In [ ]:
strec = 0
#endrec = 300  
epdur = endrec
new_events = mne.make_fixed_length_events(reconst_raw, 
                                          start=strec, stop=endrec, duration=epdur)
epochs = mne.Epochs(reconst_raw,new_events, tmin=0,
                    tmax = epdur, baseline=None)

# sh=2
# cfq = 15
# freqs = np.logspace(*np.log10([cfq-sh,cfq+sh ]), num=6)
power_afterICA = mne.time_frequency.tfr_morlet(epochs, freqs=freqs, n_cycles=n_cycles, use_fft=True,
                        return_itc=False, decim=decim, n_jobs=10)

In [ ]:
tremfreq_this_subj = gen_subj_info[sind_str]['tremfreq']
print(tremfreq_this_subj)

In [ ]:
#select some interval types and restrict by time
reg_intervals = 1
if reg_intervals:
    intlen_s = 10
    timeints = [(0,endrec,'entire')]
    int_bounds = np.arange(raw.times[0],raw.times[-1],intlen_s)
    for ib in range(1,len(int_bounds)):
        st,end = int_bounds[ib-1],int_bounds[ib]
        it = 'reg{}_{}'.format(intlen_s,ib)
        timeints += [(st,end,it)]
else:
    timeints_ = [(0,endrec,'entire')]

    for t in ['trem', 'hold', 'move', 'notrem']:
        for side in ['L', 'R']:
            it = '{}_{}'.format(t,side)
            print(it, ivalis.get(it,[]))
            timeints_ += ivalis.get(it,[])

    timeints = []
    for i,ti in enumerate(timeints_):
        timin,timax,tiname = ti
        if timin <= endrec - decim /sfreq:
            timeints += [ti]
        
print(len(timeints))

In [ ]:
rf = list( map(lambda x: x.find('_{}'.format(maintremside[0].upper()) ), anns.description) )
ann_inds = np.where(np.array(rf) >= 0)[0]


In [ ]:
for t in anns:
    print(t['onset'])

In [ ]:
print( len(timeints) )

#timeints = [ ('all',0,endrec), ('no_mvt',0,300), ('only_mvt',300,endrec) ]
fbs = [ ('tremfreq', 3,9), ('alpha/mu', 8,13),  ('beta', 15,30),  ('gamma', 30,90)  ]
fbs = [ ('tremV', tremfreq_this_subj-1,tremfreq_this_subj+1), ('tremfreq', 3,9), ('alpha/mu', 8,13),  ('low_beta', 15,22), ('high_beta', 22,30),  
       ('low_gamma', 30,60), ('high_gamma', 60,90)  ]

In [ ]:
utils.plotBandLocations(power_afterICA,\
timeints,fbs,prefix=rawname_+'_afterICA_regint{}_'.format(reg_intervals),\
anns=anns)
plt.close()

In [ ]:
%matplotlib qt

In [ ]:
tfr = power_afterICA

In [ ]:
#fig = tfr.plot_topo(vmin=mn,vmax=mx)
#%debug
fig = tfr.plot_topo(dB=False, mode='zlogratio', baseline=(None,None), vmin=-2,vmax=2)

In [ ]:
#help(power_afterICA.plot)

In [ ]:
ax2.xaxis.set

In [ ]:
#help(power_afterICA.plot)

In [ ]:
set([2, 534,3]) == set([2, 3, 534])

In [ ]:
itypes = ['trem_L','trem_R']
itypes = ['trem_L', 'hold_L']

chnames = ['MEG2633',  'MEG1442',  'MEG1342', 'MEG2323',  'MEG1942',  'MEG1613']

nr = len(chnames)
nc = 1
ww = 12; hh = 3 
fig, axs =plt.subplots(nr,nc, figsize=(nc*ww,nr*hh), sharex='row')

for i in range(nr):
    ax = axs[i]
    chname = chnames[i]
    power_afterICA.plot(picks=[chname], mode='zlogratio', 
                        baseline=(None,None), vmin=-2,vmax=2, colorbar=0, axes=ax)
    ax.set_ylabel(chname)
    lw=2

    ii = 0
    ticks = []
    ticklabels = []
    for itype in itypes:
        intervals = ivalis[itype]
        for i,iv in enumerate(intervals):
            st,end,_ = iv
    #         ax.axvline(x=st, c='purple', lw=lw)
    #         ax.axvline(x=end, c='purple', lw = lw)
            ticks += [st, end]
            ticklabels += [ '{}^'.format(itype), '{}v'.format(itype)]
            ii += 2

    if i == 0:
        ax2 = ax.twiny()  # we want to share y axis, not x! x should be different
        ax2.set_xlim(ax.get_xlim() )
        #ax2.xaxis.tick
        ax2.xaxis.set_label_position("top")
        ax2.set_xticks(ticks)
        ax2.set_xticklabels(ticklabels,rotation=90);
    

In [ ]:
len(ticklabels), len(ticks)

In [ ]:
print(ticks)

In [ ]:
strec = 0
#endrec = 300  
epdur = endrec
new_events = mne.make_fixed_length_events(filt_raw, start=strec, stop=endrec, duration=epdur)
epochs = mne.Epochs(raw,new_events, tmin=0,tmax = epdur, baseline=None)

# sh=2
# cfq = 15
# freqs = np.logspace(*np.log10([cfq-sh,cfq+sh ]), num=6)
n_cycles = freqs #/ 2.  # different number of cycle per frequency
power_noICA = mne.time_frequency.tfr_morlet(epochs, freqs=freqs, n_cycles=n_cycles, use_fft=True,
                        return_itc=False, decim=decim, n_jobs=10)

In [ ]:
utils.plotBandLocations(power_noICA,timeints,fbs,prefix=rawname_+'_beforeICA__regint{}_'.format(reg_intervals), anns=anns)
plt.close()

In [ ]:
power_afterICA.data.shape

In [ ]:
power_afterICA.info

In [ ]:
tfr = power_afterICA
picks = mne.pick_types(tfr.info, meg='mag', ref_meg=False,exclude='bads')
#timei = np.where( (tfr.times <= timax) * (tfr.times >=timin) )[0]
data = tfr.data[picks]

In [ ]:
mn,mx = np.min(data), np.max(data)

In [ ]:
mn,mx = np.min(tfr.data), np.max(tfr.data)

In [ ]:
#fig = tfr.plot_topo(vmin=mn,vmax=mx)
#%debug
#fig = tfr.plot_topo(dB=0, mode='zlogratio', baseline=(0,10))

In [ ]:
gc.collect()

In [ ]:
fig = tfr.plot_joint(dB=0, mode='percent')

In [ ]:
import functools

In [ ]:
#help(functools.partial)

In [ ]:
mne.time_frequency.tfr._prepare_picks(tfr.info, tfr.data, None, axis=0)

In [ ]:
info2,data2 = _

In [ ]:
data2.shape